## Combbine all diagnosis

This method take diagnosis from images, clinical, and diagnosis sheet, and creates one ground truth (where all three agree) and one majority vote (where two agree) diagnosis files. 

In [ ]:
import pandas as pd
import math
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
#this file is the metadata file that one can get from downloading MRI images from ADNI
img = pd.read_csv("INSERT_METADATA_FILE_HERE.csv")
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "Phase"]]

In [ ]:
def read_diagnose(file_path: str = 'DXSUM_PDXCONV_ADNIALL.csv', verbose=False):
    # Read diagnostic summary
    diagnostic_summary = pd.read_csv(file_path, index_col='PTID')
    diagnostic_summary = diagnostic_summary.sort_values(by=["update_stamp"], ascending=True)
    # Create dictionary
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        # Iterate for each row of the document
        phase: str = data['Phase']
        diagnosis: float = -1.
        if phase == "ADNI1":
            diagnosis = data['DXCURREN']
        elif phase == "ADNI2" or phase == "ADNIGO":
            dxchange = data['DXCHANGE']
            if dxchange == 1 or dxchange == 7 or dxchange == 9:
                diagnosis = 1.
            if dxchange == 2 or dxchange == 4 or dxchange == 8:
                diagnosis = 2.
            if dxchange == 3 or dxchange == 5 or dxchange == 6:
                diagnosis = 3.
        elif phase == "ADNI3":
            diagnosis = data['DIAGNOSIS']
        else:
            print(f"ERROR: Not recognized study phase {phase}")
            exit(1)
        # Update dictionary
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis
    if verbose:
        print_diagnostic_dict_summary(diagnostic_dict)
    return diagnostic_dict


def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_NL = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_NL += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
    print(f"Number of NL patients: {n_NL}\n"
          f"Number of MCI patients: {n_MCI}\n"
          f"Number of AD patients: {n_AD}\n")

In [ ]:
d = read_diagnose()
print_diagnostic_dict_summary(d)

In [ ]:
new = pd.DataFrame.from_dict(d, orient='index').reset_index()

In [ ]:
clinical.head()

In [ ]:
clinical["year"] = clinical["EXAMDATE"].str[:4]

In [ ]:
clinical["Subject"] = clinical["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")

In [ ]:
c = comb.merge(clinical, on = ["RID", "Phase"])

In [ ]:
c = c.drop("Subject", axis =1)

In [ ]:
c = c.rename(columns = {"PTID":"Subject"})

In [ ]:
img["year"] = img["Acq Date"].str[5:].str.replace("/", "")

In [ ]:
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])

In [ ]:
c["DX"] = c["DX"] -1

In [ ]:
new[0] = new[0].astype(int) -1

In [ ]:
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})

In [ ]:
m = new.merge(c, on = "Subject", how = "outer").merge(img, on = "Subject", how = "outer")

In [ ]:
m[["GroupN", "DX", "Group"]]

In [ ]:
m = m[["Subject", "GroupN", "Group", "DX", "Phase"]].drop_duplicates()

In [ ]:
m = m.dropna(subset = ["GroupN", "Group", "DX"], how="all").drop_duplicates()
m

In [ ]:
m.loc[m["DX"].isna() & m["Group"].isna(), "Group"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]
m.loc[m["DX"].isna() & m["Group"].isna(), "DX"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]

In [ ]:
m1 = m[m["GroupN"] == m["Group"]]
m3 = m[m["GroupN"] == m["DX"]]
m4 = m[m["Group"] == m["DX"]]
m2 = m1[m1["Group"] == m1["DX"]]

In [ ]:
m1 = m1[["Subject", "GroupN", "Group", "DX", "Phase"]]
m1

In [ ]:
m1.loc[m1["DX"].isna(), "DX"] = m1.loc[m1["DX"].isna(), "Group"]

In [ ]:
m3 = m3[["Subject", "GroupN", "Group", "DX", "Phase"]]
m3

In [ ]:
m3.loc[m3["Group"].isna(), "Group"] = m3.loc[m3["Group"].isna(), "GroupN"]

In [ ]:
m4 = m4[["Subject", "GroupN", "Group", "DX", "Phase"]]
m4

In [ ]:
m4[m4["GroupN"] != m4["DX"]]

In [ ]:
m2[["Subject", "GroupN", "Group", "DX", "Phase"]]

In [ ]:
m5 = pd.concat([m1,m3,m4])
i = m5[m5["Group"] == m5["GroupN"]]
i = i[i["Group"] == i["DX"]]

In [ ]:
i = i.drop_duplicates()

In [ ]:
i

In [ ]:
i[["Subject", "Group", "Phase"]].to_csv("ground_truth.csv")

In [ ]:
m.update(m5[~m5.index.duplicated(keep='first')])

In [ ]:
indexes = m.index

In [ ]:
#if none of the three diagnosis agree, then we set the value to -1
m["GROUP"] = -1

In [ ]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["Group"]):
        val = row["GroupN"]
        
        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["DX"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val
        
    elif (row["Group"] == row["DX"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val
        

In [ ]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

In [ ]:
m[m["GROUP"] != -1]

In [ ]:
m[["Subject", "GroupN", "Group", "DX", "GROUP", "Phase"]].to_csv("diagnosis_full.csv")